# Exploratory Data Analysis (EDA)
## Proyek: Prediksi Musiman Penjualan Ayam – Kios Vantedjo

**Tujuan:** Memahami pola dan karakteristik data penjualan harian

**Owner:** Elfa (Data Analyst)

**Tanggal:** [29-10-2025]


In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Time Series Libraries
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading & Initial Inspection

In [ ]:
# --- 1. Data Loading ---
# Muat data mentah Anda
df = pd.read_csv('Parsing Data - Parsing.csv', header=1)
print(f"Data mentah dimuat, {df.shape[0]} baris.")
   

## 2. Data Quality Assessment

In [ ]:
# --- 2. Data Quality & Cleaning ---
# Salin-tempel kode cleaning dari notebook sebelumnya

# 2a. Memperbaiki 'Ayam Tua (A.T)'
df['Ayam Tua (A.T)'] = df['Ayam Tua (A.T)'].astype(str).str.replace(' ', '')
df['Ayam Tua (A.T)'] = pd.to_numeric(df['Ayam Tua (A.T)'], errors='coerce')

# 2b. Imputation (Mengisi Nilai Kosong)
columns_to_fill = ['Banyak Ekor', 'Ayam Potong (A.P)', 'Ayam Kampung (A.K)', 'Ayam Tua (A.T)']
df[columns_to_fill] = df[columns_to_fill].fillna(0)

# 2c. Encoding 'Bulan'
bulan_map = {'JANUARI': 1, 'FEBRUARI': 2, 'MARET': 3, 'APRIL': 4, 'MEI': 5, 'JUNI': 6}
df['Bulan'] = df['Bulan'].map(bulan_map)

print("Data selesai dibersihkan.")
df.info()

## 3. Univariate Analysis

In [ ]:
df['Tahun'] = 2024
df['date'] = pd.to_datetime(df[['Tahun', 'Bulan', 'Tanggal']].rename(columns={
    'Tahun': 'year', 'Bulan': 'month', 'Tanggal': 'day'
}))

# 3b. AGREGASI DATA
# Ini adalah langkah terpenting.
# Kita jumlahkan semua transaksi per hari.
kolom_agregasi = ['Banyak Ekor', 'Ayam Potong (A.P)', 'Ayam Kampung (A.K)', 'Ayam Tua (A.T)', 'Harga']
df_daily = df.groupby('date')[kolom_agregasi].sum()

# 3c. Mengisi Hari yang Hilang (Opsional tapi Direkomendasikan)
# Membuat rentang tanggal penuh dari awal sampai akhir pencatatan
# 'D' berarti frekuensi Harian (Daily)
full_date_range = pd.date_range(start=df_daily.index.min(), end=df_daily.index.max(), freq='D')
df_daily = df_daily.reindex(full_date_range).fillna(0)

# 3d. Set 'date' sebagai Index (Opsional, tapi praktik yang baik)
# (Sebenarnya groupby 'date' sudah membuatnya jadi index, tapi ini memastikan)
df_daily = df_daily.asfreq('D') # Memastikan frekuensi harian

print(f"\nData telah diagregasi menjadi {df_daily.shape[0]} hari.")
print("Data harian siap untuk analisis Time Series:")
df_daily.head()

## 4. Time Series Analysis

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Kita fokus pada 'Harga'.
# period=7 karena kita ingin melihat pola mingguan (7 hari)
decomposition = seasonal_decompose(df_daily['Harga'], model='additive', period=7)

fig = decomposition.plot()
fig.set_size_inches(12, 8)
plt.suptitle('Dekomposisi Time Series (Pola Mingguan)', y=1.02)
plt.show()

## 5. Calendar Effects Analysis

In [ ]:

# Buat kolom baru untuk hari dalam seminggu
df_daily['day_of_week'] = df_daily.index.day_name()

# Tampilkan pola penjualan berdasarkan hari
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_daily, x='day_of_week', y='Harga')
plt.title('Distribusi Penjualan Harian berdasarkan Hari')
plt.xlabel('Hari')
plt.ylabel('Total Penjualan Harian (Rp)')
plt.show()

## 6. Summary & Insights

### Key Findings:
- Pola musiman mingguan (7-harian) menunjukkan pola yang kuat dan dapat diprediksi. Penjualan sangat bergantung pada hari dalam seminggu, dengan puncak penjualan yang secara konsisten terjadi pada hari Jumat dan Sabtu.
- Analisis tren jangka panjang mengindikasikan bahwa bisnis ini mengalami pertumbuhan yang lambat namun stabil selama periode 6 bulan. Rata-rata penjualan harian meningkat secara bertahap dari Januari hingga Juni.
- Dari perspektif transaksi individu, bisnis ini didominasi oleh banyak transaksi bernilai kecil (ditunjukkan oleh penumpukan data di sisi kiri distribusi). Namun, terdapat outlier berupa transaksi bernilai sangat tinggi yang jarang terjadi, kemungkinan berupa pesanan grosir atau pesanan besar.
- Kombinasi antara tren yang jelas dan pola musiman yang sangat kuat (dibuktikan oleh lonjakan pada lag 7, 14, dan 21 dalam grafik ACF) membuat data ini sangat cocok untuk peramalan. Meskipun demikian, data ini bersifat non-stasioner (akibat adanya tren) sehingga memerlukan penanganan teknis sebelum dilakukan pemodelan.

### Recommendations for Modeling:
- Rekomendasi 1: Gunakan Model yang Menangani Musiman (SARIMA)
Model ARIMA standar tidak cukup karena tidak dapat menangkap pola musiman yang kuat, seperti lonjakan penjualan pada hari Jumat dan Sabtu. SARIMA (Seasonal ARIMA) dirancang khusus untuk data dengan siklus berulang, sehingga lebih sesuai untuk meningkatkan akurasi prediksi.
Aksi: Atur parameter musiman (m atau seasonal period) menjadi 7 untuk merepresentasikan pola mingguan. Ini akan membantu model mengenali dan memprediksi fluktuasi harian secara efektif.
- Rekomendasi 2: Stabilkan Data dengan Differencing
Data ini non-stasioner akibat tren naik secara bertahap dari Januari hingga Juni, yang dapat menyebabkan model gagal dalam memprediksi pola jangka panjang. Differencing adalah teknik sederhana untuk menghilangkan tren ini dan membuat data lebih stabil (stasioner), sehingga model dapat bekerja optimal.
Aksi: Terapkan differencing non-musiman dengan parameter d=1 (atau d=2 jika tren masih kuat setelah pengujian). Gunakan uji seperti Augmented Dickey-Fuller (ADF) untuk memverifikasi bahwa data telah menjadi stasioner setelah differencing.
- Rekomendasi 3: Integrasikan Efek Kalender dengan SARIMAX
Pola penjualan yang bergantung pada hari dalam seminggu (misalnya, puncak pada Jumat dan Sabtu) adalah efek kalender yang dapat dimanfaatkan sebagai informasi tambahan. SARIMAX (SARIMA with eXogenous variables) memungkinkan penambahan faktor eksternal ini, yang akan meningkatkan presisi prediksi secara signifikan dibandingkan model dasar.
Aksi: Gunakan SARIMAX dan tambahkan variabel dummy (regressor) untuk setiap hari dalam seminggu, seperti is_Friday=1 (jika hari Jumat) atau 0 (selainnya), serta serupa untuk Sabtu dan hari lainnya. Ini memungkinkan model untuk "belajar" pengaruh spesifik hari tersebut terhadap penjualan.
- Rekomendasi 4: Identifikasi dan Tangani Anomali (Outlier)
Grafik dekomposisi menunjukkan lonjakan residual yang tidak dapat dijelaskan oleh tren atau musiman, seperti pesanan besar satu kali (misalnya, katering atau libur spesial). Anomali ini dapat mengganggu pembelajaran model jika tidak ditangani, sehingga penting untuk membersihkannya agar prediksi lebih andal.
Aksi: Lakukan investigasi manual terhadap lonjakan tersebut dengan memeriksa data historis (misalnya, tanggal spesifik). Jika anomali bersifat satu kali dan tidak berulang, bersihkan dengan mengganti nilai ekstrem menggunakan metode seperti imputasi rata-rata atau median dari data sekitarnya. Setelah itu, uji ulang model untuk memastikan peningkatan akurasi.